In [2]:
# Import modules
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier

/home/mgn/miniconda3/envs/dsml-py3.9/lib/python3.9/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.7.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
/home/mgn/miniconda3/envs/dsml-py3.9/lib/python3.9/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.2' currently installed).
  from pandas.core import (


In [3]:
# Download dataset from url, save it to sonar.csv
!wget -O sonar.csv https://archive.ics.uci.edu/ml/machine-learning-databases/undocumented/connectionist-bench/sonar/sonar.all-data

--2024-09-18 10:15:30--  https://archive.ics.uci.edu/ml/machine-learning-databases/undocumented/connectionist-bench/sonar/sonar.all-data
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified
Saving to: ‘sonar.csv’

sonar.csv               [  <=>               ]  85,72K   214KB/s    in 0,4s    

2024-09-18 10:15:36 (214 KB/s) - ‘sonar.csv’ saved [87776]



In [8]:
# Load data into a pandas dataframe and take a quick look
data = pd.read_csv("sonar.csv", header=None)
print("Shape:", data.shape)
data.head(5)

Shape: (208, 61)


,0,1,2,3,4,5,6,7,8,9,...,51,52,53,54,55,56,57,58,59,60
0,0.0200,0.0371,0.0428,0.0207,0.0954,0.0986,0.1539,0.1601,0.3109,0.2111,...,0.0027,0.0065,0.0159,0.0072,0.0167,0.0180,0.0084,0.0090,0.0032,R
1,0.0453,0.0523,0.0843,0.0689,0.1183,0.2583,0.2156,0.3481,0.3337,0.2872,...,0.0084,0.0089,0.0048,0.0094,0.0191,0.0140,0.0049,0.0052,0.0044,R
2,0.0262,0.0582,0.1099,0.1083,0.0974,0.2280,0.2431,0.3771,0.5598,0.6194,...,0.0232,0.0166,0.0095,0.0180,0.0244,0.0316,0.0164,0.0095,0.0078,R
3,0.0100,0.0171,0.0623,0.0205,0.0205,0.0368,0.1098,0.1276,0.0598,0.1264,...,0.0121,0.0036,0.0150,0.0085,0.0073,0.0050,0.0044,0.0040,0.0117,R
4,0.0762,0.0666,0.0481,0.0394,0.0590,0.0649,0.1209,0.2467,0.3564,0.4459,...,0.0031,0.0054,0.0105,0.0110,0.0015,0.0072,0.0048,0.0107,0.0094,R


In [12]:
feature_selection = range(0, 60) # All features
#feature_selection = [1, 2, 3, 4, 5] # Only a few features

# Split data into features and labels, convert labels R->0 and M->1
data.iloc[:, 60] = 1 * (data.iloc[:, 60] == 'M') # Replace labels M and R with 1 and 0
X, y = data.iloc[:, feature_selection].values, data.iloc[:, 60].values.astype(int)

# Split into train and test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [13]:
y.dtype

dtype('int64')

**Write your own boosting loop**

You should now attempt to write your own boosting classifier using `DecisionTreeClassifier` as the base classifier in an ensamble. In this example, the weighted voting of the final ensamble will be based on the *accuracy* of each single classifier. You can use the information below to fill in the missing parts of the code.

The steps of the algorithm are:

1. **Choose a weak classifier as base**   
2. Initialize weights for each sample
3. **For T rounds**
    1. Normalize the weights
    2. For available features from the set, train a classifier using a single feature and evaluate the weighted training error
    3. Choose the classifier with the lowest error
    4. **Update the weights of the training samples: increase if classified wrongly by this classifier, decrease if classified correctly**
4. Form the final strong classifier as the linear combination of the T classifiers with coefficient correlating to how good they are as individual classifiers

We will use decision-trees with `max_depth=1` as the week classifiers. These can be constructed with the `DecisionTreeClassifier` from sklearn. If `DecisionTreeClassifier` uses `max_features=None`, all features are considered for use in the split. With this setup we get steps 3.2 and 3.3 for free. Training using weights can be done using the keyword `sample_weight` when calling `fit` in sklearn.

Your task is to try out different settings for our base classifier and try different number of iterations when learning. You should also implement some update for the weight vector where if we predict correct on sample $i$ then $w_i$ should decrease while if we predict wrong $w_i$ should increase.

In [14]:
classifiers = []
coeff = []

# Initially all samples are weighted the same
w = np.ones(len(y_train))

n_iterations = 100

for i in range(1, n_iterations+1):
    clf = DecisionTreeClassifier(
        # Try different parameters to the classifier
    )

    w = w / np.sum(w) # Normalize weights
    clf.fit(X_train, y_train, sample_weight=w)

    w =  # Use suitable update based on correct/incorrect classifications on training data

    classifiers.append(clf)
    coeff.append(clf.score(X_train, y_train)) # We use accuracy as ensamble coefficient

    # Find the ensamble prediction and evaluate the accuracy after adding each new classifier
    y_hat = sum(coeff[i] * classifiers[i].predict(X_test) for i in range(len(coeff))) / sum(coeff)
    score = np.mean(np.round(y_hat) == y_test)
    print("{}-classifier ensamble: test accuracy {}".format(i, score))

SyntaxError: invalid syntax (3766071600.py, line 17)

**Compare to `AdaBoostClassifier`**

We evaluate an `AdaBoostClassifier` in a similar fashion and print the error for each number of classifiers as in the previous cell. Do you get similar performance for both of them?

In [ ]:
for i in range(1, n_iterations+1):
    clf = AdaBoostClassifier(n_estimators=i,learning_rate=1)
    clf.fit(X_train, y_train)
    score = clf.score(X_test, y_test)
    print("{} classifiers: test accuracy {}".format(i, score))